In [1]:
import sqlite3, datetime, time, json, queue, threading, sys, os, asyncio

with open('secret.json') as fr: sec = json.load(fr)
# sec = {'bulb_ip':"192.168.x.x", 'ac_ip':"192.168.x.x", 'gateway_ip':"192.168.x.x"}

In [2]:
def initialize_database(): 
    con = sqlite3.connect(f'SHT40.{datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%dT%H:%M:%S")}.db')  
    cur = con.cursor() 
    cur.execute('CREATE TABLE IF NOT EXISTS log  (Temperature REAL, Humidity REAL, Photoresistor INTEGER, timestamp REAL);')
    con.commit() 
    con.close() 
#initialize_database()

In [3]:
def database(): 
    con = sqlite3.connect('SHT40.2021-06-27T18:51:14.db')  
    cur = con.cursor() 
    while True: 
        opt, foo = q_in.get() # q.get() = opt_foo
        bar = False           # opt is a placeholder 
        try:
            if opt == 'insert':
                cur.execute('INSERT INTO log VALUES (?, ?, ?, ?);', 
                            (foo['Temperature (C)'], foo['Humidity (% rH)'], foo['Photoresistor'], time.time(),))  
            elif opt == 'sql':
                cur.execute(foo['sql'])  
                bar = cur.fetchall()
            elif opt == 'commit': 
                con.commit() 
                bar = 'Database commited'
                print('Database commited')
            elif opt == 'close': 
                con.commit()
                con.close()
                bar = 'Database commited and closed'
                print('Database commited and closed')
        except Exception as e:
            print('sql_exception', e, opt, foo)
            pass
        q_out.put(bar) 
        q_out.join()
        q_in.task_done()

q_in, q_out = queue.Queue(), queue.Queue() 
threading.Thread(target=database, name='thread-1', daemon=True).start() 

def database_operator(opt_foo): 
    q_in.join()        
    q_in.put(opt_foo)  
    bar = q_out.get()  
    q_out.task_done()   
    return bar

In [4]:
sys.path.append(f"{os.environ['HOME']}/py3/lib/python3.8/site-packages/") 
import kasa 

class HUB:
    def __init__(self):
        self.ac     = kasa.SmartPlug(sec['ac_ip'])
        self.bulb   = kasa.SmartBulb(sec['bulb_ip']) 
        self.ac_flag   = False
        self.bulb_flag = False
        self.bulb_auto = True # Change it to false for manual mode. Why manual mode?
                              # I want the light be turned on automatically at dusk but I also need to turn it off at night.
        threading.Thread(target=self.updater).start() 
            
    def updater(self):
        while 1:
            asyncio.run(self.bulb.update()) 
            asyncio.run(self.ac.update())
            if self.bulb.is_on:
                self.bulb_flag = True
            else:   
                self.bulb_flag = False 
            if self.ac.is_on: 
                self.ac_flag = True
            else:  
                self.ac_flag = False
            time.sleep(180)
hub = HUB()

In [ ]:
import flask
app = flask.Flask(__name__) 

import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

temp_ceil  = 26.5
temp_floor = 25.5
lumi_ceil  = 350
lumi_floor = 200

@app.route("/", methods=['GET'])
def index(): 
    return 'Hello~' 

@app.route("/bulb_auto", methods=['GET', 'POST'])
def bulb_auto():
    req = flask.request
    if req.method == 'GET':
        return {'status':hub.bulb_auto}
    elif req.method == 'POST': 
        hub.bulb_auto = flask.request.json['status']
        return {'status':hub.bulb_auto}
    
@app.route("/log", methods=['POST'])
def log():
    T = flask.request.json['Temperature (C)']
    P = flask.request.json['Photoresistor']
    
    if T > temp_ceil and hub.ac_flag == False: 
        asyncio.run(hub.ac.turn_on()) 
        hub.ac_flag = True
    elif T < temp_floor and hub.ac_flag == True:
        asyncio.run(hub.ac.turn_off()) 
        hub.ac_flag = False
        
    if P > lumi_ceil and hub.bulb_flag == True: 
        asyncio.run(hub.bulb.turn_off())  
        hub.bulb_flag = False
    elif hub.bulb_auto == True and P < lumi_floor and hub.bulb_flag == False:
        asyncio.run(hub.bulb.turn_on())     
        hub.bulb_flag = True     
        
    database_operator(('insert', flask.request.json)) 
    return '' 

@app.route("/sql", methods=['POST'])
def sql():  
    return {'result':database_operator(('sql', flask.request.json)).__repr__() }

@app.route('/heartbeat', methods=['GET']) 
def heartbeat():
    database_operator(('commit', None))  
    return 'Cheers!'

app.run(sec['gateway_ip'],'4433') 
# database_operator(('sql', {'sql':'select * from log'})) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [ ]:
database_operator(('commit', ''))